# Estimating counterfactuals

## Reasoning model

In [1]:
from pybbn.probabilistic import create_reasoning_model

def get_model():
    d = {
        'nodes': ['drug', 'gender', 'recovery'],
        'edges': [('gender', 'drug'), ('gender', 'recovery'), ('drug', 'recovery')]
    }
    
    p = {
        'gender': {
            'columns': ['gender', '__p__'],
            'data': [
                ['male', 0.51], ['female', 0.49]
            ]
        },
        'drug': {
            'columns': ['gender', 'drug', '__p__'],
            'data': [
                ['female', 'no', 0.24],
                ['female', 'yes', 0.76],
                ['male', 'no', 0.76],
                ['male', 'yes', 0.24]
            ]
        },
        'recovery': {
            'columns': ['gender', 'drug', 'recovery', '__p__'],
            'data': [
                ['female', 'no', 'no', 0.90],
                ['female', 'no', 'yes', 0.10],
                ['female', 'yes', 'no', 0.27],
                ['female', 'yes', 'yes', 0.73],
                ['male', 'no', 'no', 0.99],
                ['male', 'no', 'yes', 0.01],
                ['male', 'yes', 'no', 0.07],
                ['male', 'yes', 'yes', 0.93]
            ]
        }
    }
    
    model = create_reasoning_model(d, p)
    return model

model = get_model()

## Data simulation

In [2]:
N = 10_000
Xy = model.sample(max_samples=N)
Xy.shape

(10000, 3)

In [3]:
Xy.head()

,gender,drug,recovery
0,male,no,no
1,male,yes,yes
2,female,yes,no
3,male,yes,yes
4,female,yes,yes


## Learn hidden node

In [4]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

def learn_hidden_node(n_clusters, Xy):
    X = pd.get_dummies(Xy).map(int)

    if n_clusters == 'auto':
        u_records = X.drop_duplicates().shape[0]
        u_values = Xy['recovery'].unique().shape[0]
        n_clusters = (u_records + u_values) / 2.0
        n_clusters = np.round(n_clusters)
        n_clusters = int(n_clusters)
        
    n_unique = X.drop_duplicates().shape[0]
    if n_clusters > n_unique:
        n_clusters = n_unique

    n_unique = Xy['recovery'].unique().shape[0]
    if n_clusters < n_unique:
        n_clusters = n_unique
    
    k = KMeans(n_init='auto', n_clusters=n_clusters, max_iter=1_000, random_state=37).fit(X)
    c = k.predict(X)
    
    return Xy.assign(**{'__u__': [f'c{label}' for label in c]})

Xy = learn_hidden_node('auto', Xy)
Xy.shape

(10000, 4)

In [6]:
Xy.head()

,gender,drug,recovery,__u__
0,male,no,no,c1
1,male,yes,yes,c0
2,female,yes,no,c3
3,male,yes,yes,c0
4,female,yes,yes,c0


## Build SCM

In [7]:
import itertools

def get_cpt_with_parents(y, parents, df):
    pa_domains = {pa: sorted(list(df[pa].unique())) for pa in parents}
    y_domain = sorted(list(df[y].unique()))

    N = df.shape[0]
    data = []
    for pa_v in itertools.product(*[pa_domains[pa] for pa in parents]):
        for y_v in y_domain:
            q_pa = ' and '.join([f'{_pa}=="{_pa_v}"' for _pa, _pa_v in zip(parents, pa_v)])
            q_joint = f'{q_pa} and {y}=="{y_v}"'

            p_pa = df.query(q_pa).shape[0]
            p_joint = df.query(q_joint).shape[0]

            p = p_joint / p_pa if p_pa != 0 else 0

            row = list(pa_v) + [y_v, p]
            data.append(row)

    return {
        'columns': parents + [y, '__p__'],
        'data': data
    }

def get_cpt_without_parents(y, df):
    n = df[y].value_counts().sort_index()
    p = n / df.shape[0]
    return {
        'columns': [y, '__p__'],
        'data': [[_i, _v] for _i, _v in zip(p.index, p.values)]
    }

def get_cpt(y, parents, df):
    if len(parents) == 0:
        return get_cpt_without_parents(y, df)
    else:
        return get_cpt_with_parents(y, parents, df)

def get_scm_parameters(y, parents, df, model):
    p = {p: model.node_potentials[p].to_dict('split', index=False) for p in parents}
    p['__u__'] = get_cpt('__u__', [], df)
    p[y] = get_cpt(y, parents + ['__u__'], df)

    return p

def get_scm_graph(y, parents, model):
    d = model.d.copy()

    include_nodes = set(parents + [y])
    all_nodes = set(d.nodes())
    for n in all_nodes:
        if n not in include_nodes:
            d.remove_node(n)
    
    d.add_node('__u__')
    d.add_edge('__u__', y)
    
    return d

def get_scm(y, parents, Xy, model):
    d = get_scm_graph(y, parents, model)
    p = get_scm_parameters(y, parents, Xy, model)
    m = create_reasoning_model(d, p)
    return m

scm_model = get_scm('recovery', ['gender', 'drug'], Xy, model)

## Abduction

In [8]:
_e = scm_model.get_observation_evidences({'gender': 'male', 'drug': 'yes', 'recovery': 'yes'})
u = scm_model.pquery(['__u__'], evidences=_e)['__u__'] \
    .assign(**{'__p__': lambda d: d['__p__'] + 0.001}) \
    .assign(**{'__p__': lambda d: d['__p__'] / d['__p__'].sum()})
u

,__u__,__p__
0,c0,0.996020
1,c1,0.000995
2,c2,0.000995
3,c3,0.000995
4,c4,0.000995


## Prediction

In [9]:
_e = scm_model.get_observation_evidences({'gender': 'male', 'drug': 'no'})
_e['__u__'] = u

scm_model.pquery(['recovery'], evidences=_e)['recovery']

,recovery,__p__
0,no,0.603564
1,yes,0.396436
